In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, EvalPrediction
from datasets import load_dataset, DatasetDict
import torch
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef
)

In [3]:
model_ckpt = "Alibaba-NLP/gte-base-en-v1.5"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print(device)

cuda


In [8]:
dataset = DatasetDict({
    "train": load_dataset("csv", data_files="/content/drive/MyDrive/Automatic-exam-scoring/data/train_oversampled.csv", split="train"),
    "validation": load_dataset("csv", data_files="/content/drive/MyDrive/Automatic-exam-scoring/data/val_oversampled.csv", split="train"),
    "test": load_dataset("csv", data_files="/content/drive/MyDrive/Automatic-exam-scoring/data/test.csv", split="train"),
})

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'corrected_text', 'length', 'ratio_err', 'labels', 'is_generated'],
        num_rows: 14803
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'corrected_text', 'length', 'ratio_err', 'labels', 'is_generated'],
        num_rows: 3701
    })
    test: Dataset({
        features: ['corrected_text', 'length', 'ratio_err', 'labels'],
        num_rows: 3462
    })
})


In [10]:
dataset = dataset.map(
    lambda x: {
        "text" : x["corrected_text"]
    },
    remove_columns = ["corrected_text", "length", "ratio_err"]
)

Map:   0%|          | 0/14803 [00:00<?, ? examples/s]

Map:   0%|          | 0/3701 [00:00<?, ? examples/s]

In [11]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'labels', 'is_generated', 'text'],
        num_rows: 14803
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'labels', 'is_generated', 'text'],
        num_rows: 3701
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 3462
    })
})


In [12]:
dataset["train"] = dataset["train"].remove_columns(["Unnamed: 0", "is_generated"])
dataset["validation"] = dataset["validation"].remove_columns(["Unnamed: 0", "is_generated"])

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [14]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True)

In [15]:
dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/14803 [00:00<?, ? examples/s]

Map:   0%|          | 0/3701 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [16]:
tokenizer.tokenize("Hello world, how are u ?", add_special_tokens=True)

['[CLS]', 'hello', 'world', ',', 'how', 'are', 'u', '?', '[SEP]']

In [17]:
tokenizer.pad_token

'[PAD]'

In [18]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
dataset = dataset.map(lambda x: {"labels": x["labels"] - 1})

Map:   0%|          | 0/14803 [00:00<?, ? examples/s]

Map:   0%|          | 0/3701 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [20]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt,
                                                           trust_remote_code=True,
                                                           num_labels=6).to(device)

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling.py:   0%|          | 0.00/57.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

Some weights of NewForSequenceClassification were not initialized from the model checkpoint at Alibaba-NLP/gte-base-en-v1.5 and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
print(model)

NewForSequenceClassification(
  (new): NewModel(
    (embeddings): NewEmbeddings(
      (word_embeddings): Embedding(30528, 768, padding_idx=0)
      (rotary_emb): NTKScalingRotaryEmbedding()
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): NewEncoder(
      (layer): ModuleList(
        (0-11): 12 x NewLayer(
          (attention): NewAttention(
            (qkv_proj): Linear(in_features=768, out_features=2304, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (o_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (mlp): NewGatedMLP(
            (up_gate_proj): Linear(in_features=768, out_features=6144, bias=False)
            (down_proj): Linear(in_features=3072, out_features=768, bias=True)
            (act_fn): GELUActivation()
            (hidden_dropout): Dropout(p=0.1, inplace=False)
          )
          (attn_ln): LayerNorm((768,), 

In [22]:
def compute_metrics(preds: EvalPrediction):
    """
    Compute metrics for the task

    Args:
        preds {EvalPrediction}: the predictions from the model
    Returns:
        dict: a dictionary of metrics
    """

    labels = preds.label_ids
    preds = preds.predictions.argmax(-1)

    acc = accuracy_score(labels, preds)
    weighted_precision = precision_score(labels, preds, average="weighted")
    weighted_recall = recall_score(labels, preds, average="weighted")
    weighted_f1 = f1_score(labels, preds, average="weighted")
    mcc = matthews_corrcoef(labels, preds)

    macro_precision = precision_score(labels, preds, average="macro")
    macro_recall = recall_score(labels, preds, average="macro")
    macro_f1 = f1_score(labels, preds, average="macro")

    micro_precision = precision_score(labels, preds, average="micro")
    micro_recall = recall_score(labels, preds, average="micro")
    micro_f1 = f1_score(labels, preds, average="micro")

    return {
        "accuracy": acc,
        "weighted_precision": weighted_precision,
        "weighted_recall": weighted_recall,
        "weighted_f1": weighted_f1,
        "mcc": mcc,
        "macro_precision": macro_precision,
        "macro_recall": macro_recall,
        "macro_f1": macro_f1,
        "micro_precision": micro_precision,
        "micro_recall": micro_recall,
        "micro_f1": micro_f1
    }

In [23]:
BATCH_SIZE = 2
LR = 2e-5

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/models_lazy_teacherv2",
    num_train_epochs=6,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=100,
    learning_rate=LR,
    evaluation_strategy="steps",
    bf16=True,
    eval_steps=250,
    report_to="tensorboard",
    lr_scheduler_type="cosine"
)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [25]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Weighted Precision,Weighted Recall,Weighted F1,Mcc,Macro Precision,Macro Recall,Macro F1,Micro Precision,Micro Recall,Micro F1
250,0.933700,0.644573,0.736828,0.771200,0.736828,0.733244,0.693926,0.769046,0.739234,0.733520,0.736828,0.736828,0.736828
500,0.623600,0.634460,0.737098,0.773110,0.737098,0.741470,0.691023,0.771063,0.740671,0.742073,0.737098,0.737098,0.737098
750,0.554500,0.545545,0.769252,0.781455,0.769252,0.765636,0.727560,0.779716,0.771198,0.765714,0.769252,0.769252,0.769252
1000,0.499700,0.551899,0.780600,0.798666,0.780600,0.779615,0.741482,0.796563,0.782819,0.779536,0.780600,0.780600,0.780600
1250,0.483900,0.574825,0.762226,0.770122,0.762226,0.760337,0.717614,0.768286,0.763874,0.760090,0.762226,0.762226,0.762226
1500,0.453800,0.564466,0.765199,0.779200,0.765199,0.763848,0.723097,0.776993,0.767544,0.763595,0.765199,0.765199,0.765199
1750,0.515800,0.536981,0.777628,0.796114,0.777628,0.782026,0.735541,0.794017,0.779781,0.781927,0.777628,0.777628,0.777628
2000,0.359800,0.587032,0.779789,0.793070,0.779789,0.780803,0.738903,0.791306,0.781753,0.780640,0.779789,0.779789,0.779789
2250,0.389800,0.586419,0.788165,0.792424,0.788165,0.788833,0.746328,0.791341,0.788384,0.788430,0.788165,0.788165,0.788165
2500,0.376400,0.575004,0.780059,0.790015,0.780059,0.781372,0.738124,0.788371,0.781573,0.781123,0.780059,0.780059,0.780059


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

TrainOutput(global_step=5550, training_loss=0.33340324131218163, metrics={'train_runtime': 3589.4881, 'train_samples_per_second': 24.744, 'train_steps_per_second': 1.546, 'total_flos': 2.9109637866485412e+16, 'train_loss': 0.33340324131218163, 'epoch': 5.998378816536071})

In [ ]:
trainer.save_model("/content/drive/MyDrive/models_lazy_teacherv2")

In [29]:
tokenizer.save_pretrained("/content/drive/MyDrive/models_lazy_teacherv2")

('/content/drive/MyDrive/models_lazy_teacherv2/checkpoint-1000/tokenizer_config.json',
 '/content/drive/MyDrive/models_lazy_teacherv2/checkpoint-1000/special_tokens_map.json',
 '/content/drive/MyDrive/models_lazy_teacherv2/checkpoint-1000/vocab.txt',
 '/content/drive/MyDrive/models_lazy_teacherv2/checkpoint-1000/added_tokens.json',
 '/content/drive/MyDrive/models_lazy_teacherv2/checkpoint-1000/tokenizer.json')

In [27]:
trainer.model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/models_lazy_teacherv2/checkpoint-1000").to(device)

The repository for /content/drive/MyDrive/models_lazy_teacherv2/checkpoint-1000 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//content/drive/MyDrive/models_lazy_teacherv2/checkpoint-1000.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The repository for /content/drive/MyDrive/models_lazy_teacherv2/checkpoint-1000 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//content/drive/MyDrive/models_lazy_teacherv2/checkpoint-1000.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [28]:
trainer.evaluate(eval_dataset=dataset["test"])

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.946972131729126,
 'eval_accuracy': 0.5921432697862508,
 'eval_weighted_precision': 0.6133258131608486,
 'eval_weighted_recall': 0.5921432697862508,
 'eval_weighted_f1': 0.5705270220560988,
 'eval_mcc': 0.4660168258689066,
 'eval_macro_precision': 0.512330312615423,
 'eval_macro_recall': 0.45922668706000863,
 'eval_macro_f1': 0.45716929102789833,
 'eval_micro_precision': 0.5921432697862508,
 'eval_micro_recall': 0.5921432697862508,
 'eval_micro_f1': 0.5921432697862508,
 'eval_runtime': 33.8845,
 'eval_samples_per_second': 102.171,
 'eval_steps_per_second': 25.557,
 'epoch': 5.998378816536071}